In [ ]:
#establishing environment

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.io import loadmat, savemat
from scipy.signal import ellip, lfilter, filtfilt, find_peaks, butter, sosfiltfilt, sosfilt
from sklearn.model_selection import train_test_split
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline

In [ ]:
#FUNCTIONS

def NumWins(x, fs, winLen, winDisp):
    return (len(x) - winLen * fs + winDisp * fs) // (winDisp * fs)


def filter_data(raw_eeg, fs=1000):
    """
    Write a filter function to clean underlying data.
    Filter type and parameters are up to you. Points will be awarded for reasonable filter type, parameters and application.
    Please note there are many acceptable answers, but make sure you aren't throwing out crucial data or adversly
    distorting the underlying data!

    Input:
      raw_eeg (samples x channels): the raw signal
      fs: the sampling rate (1000 for this dataset)
    Output:
      clean_data (samples x channels): the filtered signal
    """
    raw_eeg_t = raw_eeg.transpose()
    filtered = []

    nyq = fs / 2

    # (b, a) = ellip(4, 0.1, 40, 20/nyq, btype='lowpass')
    sos = butter(8, [0.15, 200], btype='bandpass', output='sos', fs=fs)

    for ch_data in raw_eeg_t:
        # filtered_ch = filtfilt(b, a, ch_data)
        filtered_ch = sosfiltfilt(sos, ch_data)
        filtered.append(filtered_ch)

    filtered = np.array(filtered)

    return filtered.transpose()


# line length
def LL(x):
    return np.sum(np.absolute(np.ediff1d(x)))


# energy
def E(x):
    return np.sum(x ** 2)

#RMS
def RMS(x):
    return np.sqrt(np.mean(x**2))

# area
def A(x):
    return np.sum(np.absolute(x))


# spectral amp
def spectral_amplitude(x):
    x_fft = np.fft.fft(x)
    return np.mean(x_fft)

def mean_amplitude_freq(X, fs, lF, uF):
    time_step = 1/fs
    ps = np.abs(np.fft.fft(X)) ** 2
    freqs = np.fft.fftfreq(X.size, time_step)
    mask = np.logical_and(freqs >= lF, freqs <= uF )
    avgValue = ps[mask].mean()
    return avgValue

# number of crossings (zero) - not in
def ZX(x):
    x_demean = x - np.mean(x)
    num_crossings = 0
    for i in range(1, len(x)):
        fromAbove = False
        fromBelow = False
        if x_demean[i - 1] > 0 and x_demean[i] < 0:
            fromAbove = True
        if x_demean[i - 1] < 0 and x_demean[i] > 0:
            fromBelow = True

        if fromAbove or fromBelow:
            num_crossings += 1
    return num_crossings

def MEAN(x):
    return np.mean(x)

def bandpower(x, fs, fmin, fmax):
    f, Pxx = sig.periodogram(x, fs=fs)
    ind_min = np.argmax(f > fmin) - 1
    ind_max = np.argmax(f > fmax) - 1
    return np.trapz(Pxx[ind_min: ind_max], f[ind_min: ind_max])

# gets features, load features you want calculated from here
def get_features(filtered_window, fs=1000):
    """
      Write a function that calculates features for a given filtered window.
      Feel free to use features you have seen before in this class, features that
      have been used in the literature, or design your own!

      Input:
        filtered_window (window_samples x channels): the window of the filtered ecog signal
        fs: sampling rate
      Output:
        features (channels x num_features): the features calculated on each channel for the window
    """

    filtered_window_t = filtered_window.transpose()

    features = []

    for ch in filtered_window_t:
        features.append(np.array([MEAN(ch),
                                  mean_amplitude_freq(ch, fs, 5, 15),
                                  mean_amplitude_freq(ch, fs, 20, 25),
                                  mean_amplitude_freq(ch, fs, 75, 115),
                                  mean_amplitude_freq(ch, fs, 125, 160),
                                  mean_amplitude_freq(ch, fs, 160, 175)
                                  ]))

    features = np.array(features)

    return features

# Bandpower, can try specifying the following:
# Delta: fmin = 0.5, fmax = 4
# Theta: fmin = 4, fmax = 7
# Alpha: fmin = 8, fmax = 12
# Beta: fmin = 12.5, fmax = 30
# Gamma: fmin = 25, fmax = 140
# get_windowed_feats - filters raw ecog signal and finds features

# From the paper suggestion:
# fmin = 5, fmax = 15
# fmin = 20, fmax = 25
# fmin = 75, fmax = 115
# fmin = 125, fmax = 160
# fmin = 160, fmax = 175

def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
    """
      Write a function which processes data through the steps of filtering and
      feature calculation and returns features. Points will be awarded for completing
      each step appropriately (note that if one of the functions you call within this script
      returns a bad output, you won't be double penalized). Note that you will need
      to run the filter_data and get_features functions within this function.

      Inputs:
        raw_eeg (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
        window_length: the window's length
        window_overlap: the window's overlap
      Output:
        all_feats (num_windows x (channels x features)): the features for each channel for each time window
          note that this is a 2D array.
    """

    cleaned_ecog = filter_data(raw_ecog)
    num_wins = NumWins(cleaned_ecog.transpose()[0], fs, window_length, window_overlap)
    all_feats_3d = []
    for winStart in np.arange(0, int(num_wins), 1):
        clip = cleaned_ecog[
               int(winStart * window_overlap * fs):int(winStart * window_overlap * fs + (window_length * fs))]
        all_feats_3d.append(get_features(clip))

    num_channels = len(all_feats_3d[0])
    num_features = len(all_feats_3d[0][0])

    all_feats = np.zeros([len(all_feats_3d), num_features * num_channels])

    for k in range(int(len(all_feats_3d))):
        q = flatten_list = [j for sub in all_feats_3d[k] for j in sub]
        all_feats[k, :] = q

    return np.array(all_feats)

def repeat_preds(preds, window_to_time_ratio=50):
    pred_all = []
    for row in preds:
        for i in range(window_to_time_ratio):
            pred_all.append(row)

    # For out problem, it is short 50 entries, so add the last row 50 more times
    for i in range(window_to_time_ratio):
        pred_all.append(row)
    
    return np.array(pred_all)

def interp_preds(preds, time_length):
    # N samples
    preds_sample_orig = np.arange(len(preds))
    
    # T time points
    preds_sample_target = np.arange(time_length)
    
    preds = preds.transpose()

    preds_interp = []
    
    for finger_preds in preds:
        f = interp1d(preds_sample_orig, finger_preds)
        new_preds = f(preds_sample_target)
        preds_interp.append(new_preds)
    
    preds_interp = np.array(preds_interp).transpose()
    
    return preds_interp

def spline_preds(preds, time_length):
    # N samples
    preds_sample_orig = np.arange(len(preds))
    
    # T time points
    preds_sample_target = np.linspace(0,len(preds),time_length)
    print(preds_sample_target)
    preds = preds.transpose()

    preds_interp = []
    
    for finger_preds in preds:
        f = CubicSpline(preds_sample_orig, finger_preds, bc_type='natural')
        new_preds = f(preds_sample_target)
        preds_interp.append(new_preds)
    
    preds_interp = np.array(preds_interp).transpose()
    
    return preds_interp

def compute_corr(preds, truth):
    subj_corr = []
    for i in range(5):
        finger_pred = preds.transpose()[i]
        finger_truth = truth.transpose()[i]
        subj_corr.append(pearsonr(finger_pred, finger_truth)[0])
    
    return subj_corr



In [ ]:
# CAR the signal
def CAR(raw_eeg):
    AVG = raw_eeg.mean()
    mean_mat = np.ones(raw_eeg.size).reshape(raw_eeg.shape)
    CAR = np.subtract(raw_eeg, mean_mat)
    return CAR

def get_features(filtered_window, fs=1000):
    """
      Write a function that calculates features for a given filtered window.
      Feel free to use features you have seen before in this class, features that
      have been used in the literature, or design your own!

      Input:
        filtered_window (window_samples x channels): the window of the filtered ecog signal
        fs: sampling rate
      Output:
        features (channels x num_features): the features calculated on each channel for the window
    """

    filtered_window_t = filtered_window.transpose()

    features = []

    for ch in filtered_window_t:
        features.append(np.array([MEAN(ch),
                                  LL(ch),
                                  E(ch),
                                  bandpower(ch, fs, 1, 60),
                                  bandpower(ch, fs, 60, 100),
                                  bandpower(ch, fs, 100, 200),
                                  ]))

    features = np.array(features)

    return features

# Bandpower, can try specifying the following:
# Delta: fmin = 0.5, fmax = 4
# Theta: fmin = 4, fmax = 7
# Alpha: fmin = 8, fmax = 12
# Beta: fmin = 12.5, fmax = 30
# Gamma: fmin = 25, fmax = 140
# get_windowed_feats - filters raw ecog signal and finds features

# From the paper suggestion:
# fmin = 5, fmax = 15
# fmin = 20, fmax = 25
# fmin = 75, fmax = 115
# fmin = 125, fmax = 160
# fmin = 160, fmax = 175

#from Schalks Paper
#8 to 12 
#18 to 24 
#75 to 115
#125 to 159
#159 to 175 

def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
    """
      Write a function which processes data through the steps of filtering and
      feature calculation and returns features. Points will be awarded for completing
      each step appropriately (note that if one of the functions you call within this script
      returns a bad output, you won't be double penalized). Note that you will need
      to run the filter_data and get_features functions within this function.

      Inputs:
        raw_eeg (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
        window_length: the window's length
        window_overlap: the window's overlap
      Output:
        all_feats (num_windows x (channels x features)): the features for each channel for each time window
          note that this is a 2D array.
    """

    filter_ecog = filter_data(raw_ecog)
    cleaned_ecog = CAR(filter_ecog)
    num_wins = NumWins(cleaned_ecog.transpose()[0], fs, window_length, window_overlap)
    all_feats_3d = []
    for winStart in np.arange(0, int(num_wins), 1):
        clip = cleaned_ecog[
               int(winStart * window_overlap * fs):int(winStart * window_overlap * fs + (window_length * fs))]
        all_feats_3d.append(get_features(clip))

    num_channels = len(all_feats_3d[0])
    num_features = len(all_feats_3d[0][0])

    all_feats = np.zeros([len(all_feats_3d), num_features * num_channels])

    for k in range(int(len(all_feats_3d))):
        q = flatten_list = [j for sub in all_feats_3d[k] for j in sub]
        all_feats[k, :] = q

    return np.array(all_feats)

In [ ]:
def normalize_features(all_feats_train, all_feats_test, num_features):
    # Input should be (num_windows x (channels x features))
    # Num features is the number of unique features that were extracted
    all_feats_train_norm = np.copy(all_feats_train)
    all_feats_test_norm = np.copy(all_feats_test)
    
    feats_avg = []
    feats_std = []
    
    for n in range(num_features):
        feats_idx_train = np.arange(n,len(all_feats_train.transpose()),num_features)
        
        feat_data_train = all_feats_train[:][:,feats_idx_train]
        
        feat_means_train = np.mean(feat_data_train,axis=0)
        feat_stds_train = np.std(feat_data_train,axis=0)
        
        all_feats_train_norm[:][:,feats_idx_train] = (feat_data_train - feat_means_train)/feat_stds_train
        
        # Note that we must use the same mean and std. dev from the TRAINING set
        #     because regression models are sensitive to value domain as they are
        #     scale-variant.
        feats_idx_test = np.arange(n,len(all_feats_test.transpose()),num_features)
        feat_data_test = all_feats_test[:][:,feats_idx_test]
        all_feats_test_norm[:][:,feats_idx_test] = (feat_data_test - feat_means_train)/feat_stds_train
        
        # Sanity checking plot, comment out if you don't want plots
#         if n == 0:
#             plt.figure()
#             plt.plot(feat_data_train.transpose()[0])
#             plt.figure()
#             plt.plot(all_feats_train_norm[:][:,feats_idx_train].transpose()[0])
            
#             plt.figure()
#             plt.plot(feat_data_test.transpose()[0])
#             plt.figure()
#             plt.plot(all_feats_test_norm[:][:,feats_idx_test].transpose()[0])

    return all_feats_train_norm, all_feats_test_norm


# NOTE: SPECIFY THIS MANUALLY, 
#       it's difficult to backwards engineer the feature number after flattening
num_features = 7 # !!!!!!!!!!!!